### 로직

1. 리뷰 preproecess하고 선별
2. 선별된 리뷰 document로 뭉침
3. 키버트 돌림
4. 감성 분석 돌림
5. 최종 랭킹을 메뉴판과 매칭

### Setting

In [1]:
import pandas as pd

In [2]:
path = "./data/"

rst = pd.read_csv(path + "restaurant.csv", encoding='utf-8-sig')
menu = pd.read_csv(path + "menu.csv", encoding='utf-8-sig')
rev = pd.read_csv(path + "review.csv", encoding='utf-8-sig')

In [3]:
rst.head(3)

,restaurant_id,address,name,opening_hours,phone,type,category,lat,lng,review_count,created_at,updated_at,summary
0,0,서울 광진구 광장로 58-1 1층,범가,[],010-3627-7041,0,중식당,NaN,NaN,NaN,NaN,NaN,NaN
1,1,서울 광진구 워커힐로 177 워커힐 호텔앤리조트,비스타 워커힐 서울 피자힐,[],02-6330-9020,0,피자,NaN,NaN,NaN,NaN,NaN,NaN
2,2,서울 광진구 워커힐로 177 비스타 워커힐 서울 1층,비스타 워커힐 서울 더뷔페,[],02-6330-9015,0,뷔페,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
rst['restaurant_id'].nunique()

3165

In [5]:
menu.head(3)

,menu_id,price,restaurant_id,name,ranking,created_at,updated_at,image_url
0,0,45000,4,새조개 샤브샤브,NaN,NaN,NaN,NaN
1,1,45000,4,하모 샤브샤브,NaN,NaN,NaN,NaN
2,2,10000,4,매생이굴국밥,NaN,NaN,NaN,NaN


In [6]:
menu['restaurant_id'].nunique()

2029

In [7]:
rev.head(3)

,review_id,created_at,user_id,restaurant_id,content,updated_at
0,0,4.2.화,0,0,✨마늘후레이크 찹쌀탕수육 맛보러 가야하는 범가 ✨ 지인이 여기 탕수육 맛있다고 해서...,NaN
1,1,3.11.월,1,0,광나루역맛집 범가에서 마늘찹쌀탕수육 해물짬뽕 해물짜장 먹고 왔어요! 양도 정말 푸짐...,NaN
2,2,2.9.금,2,0,<스티브 추천> 마늘 후레이크 탕수육이 유명하죠. 근데 이번에 알았어요. 여기 ...,NaN


In [25]:
rev['restaurant_id'].nunique()

2656

### 샘플 데이터 뽑기

In [28]:
val_ = rev['restaurant_id'].value_counts()
unique_counts = val_.value_counts()

# 카운트가 40 이상인 60이하만
selected_values = val_[(val_ >= 40) & (val_ <= 60)].index

# 필터링된 값들을 포함하는 원본 데이터프레임 필터링
rev_sample = rev[rev['restaurant_id'].isin(selected_values)]

In [29]:
rev_sample['restaurant_id'].nunique()

137

In [30]:
# rev_sample_df에서 restaurant_id 열의 고유값 추출
unique_restaurant_ids = rev_sample['restaurant_id'].unique()

# menu_df에서 rev_sample_df에 있는 restaurant_id만 남기기
menu_sample = menu[menu['restaurant_id'].isin(unique_restaurant_ids)]

In [36]:
menu_sample['restaurant_id'].nunique()

92

In [34]:
sample = pd.merge(menu_sample, rev_sample, on='restaurant_id', how='inner')

In [42]:
sample.head(2)

,menu_id,price,restaurant_id,name,ranking,created_at_x,updated_at_x,image_url,review_id,created_at_y,user_id,content,updated_at_y
0,492,10000,54,판모밀,NaN,NaN,NaN,NaN,17064,3.15.금,10306,1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요!!!! 아니 회뿐...,NaN
1,492,10000,54,판모밀,NaN,NaN,NaN,NaN,17065,3.19.화,10307,아이들없이 신랑하고 둘만 일식 먹으려고 방문했는데 메뉴 나올때마다 너무 맛있어서 애...,NaN


In [40]:
sample['restaurant_id'].nunique()

92

In [41]:
menu_sample.to_csv(path + "menu_sample.csv", encoding='utf-8-sig')
rev_sample.to_csv(path + "review_sample.csv", encoding='utf-8-sig')

### functions

In [8]:
# 리뷰 테이블 받아서 필요한 리뷰만 선별해 리스트(?) 키버트의 인풋 형식으로 만드는 함수
import re
from kiwipiepy import Kiwi

def drop_short_words(df, threshold):
    # 텍스트가 없는 널값 제거
    df = df.dropna(subset=['content'])

    # 한 단어 이상인 것들만 남기기
    df['word_counts'] = df['content'].apply(lambda x: len(x.split()))

    # 한 단어(threshold) 이상인 것들만 필터링하여 새로운 데이터프레임 반환. 필요없는 word_counts는 삭제
    return df[df['word_counts'] > threshold].drop(columns=['word_counts'])

def preprocess_emoji(content):
    filtered_content = re.sub(r'[^\s\wㄱ-힣\d]', '', content)
    
    return filtered_content

kiwi = Kiwi()
def kiwi_to_sentences(combined_review):

    sentences = kiwi.split_into_sents(combined_review)

    # 출력 형식 중 텍스트만 리스트에 저장
    text_list = [sentence.text for sentence in sentences]
    
    return text_list

def review_table2input(review_table):
    # 필요 컬럼만 선별
    review_table = review_table[['restaurant_id', 'content']]

    # 2단어 이상인 리뷰만 선별 (2단어인 이유는 중간발표 appendix. word count 히스토그램 참고)
    dropped_review_table = drop_short_words(review_table, 1)

    # 'review_text' 열에 정규 표현식 함수 적용
    dropped_review_table['content'] = dropped_review_table['content'].apply(preprocess_emoji)

    # restaurant_id로 그룹화하여 content(review_text)들을 온점 단위로 합치기
    grouped_reviews = dropped_review_table.groupby('restaurant_id')['content'].apply(lambda x: ' . '.join(x)).reset_index()

    # 새로운 데이터프레임 생성 및 문장 단위 분리해 리스트 저장
    new_df = pd.DataFrame(grouped_reviews)
    new_df['review_sentence_split'] = new_df['content'].apply(kiwi_to_sentences)

    return new_df

In [22]:
path = "./data/"

menu_sample = pd.read_csv(path + "menu_sample.csv", encoding='utf-8-sig')
rev_sample = pd.read_csv(path + "review_sample.csv", encoding='utf-8-sig')

rev2input = review_table2input(rev_sample)

C:\Users\MOONSOO\AppData\Local\Temp\ipykernel_17948\3007086783.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_counts'] = df['content'].apply(lambda x: len(x.split()))


In [25]:
rev2input[:2]

,restaurant_id,content,review_sentence_split
0,54,1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요 아니 회뿐만 아니...,"[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 아니 회뿐만 ..."
1,59,자주 가는 맛집에요 고기 두툼하고 퀄리티 좋구요 정 말 맛잇어요 메뉴 다양한데 진짜...,"[자주 가는 맛집에요 고기 두툼하고 퀄리티 좋구요, 정 말 맛잇어요 메뉴 다양한데 ..."


In [18]:
# 메뉴 문장 추출

# 메뉴 데이터에서 메뉴 리스트를 먼저 만들어줍니다
# 추후 이 데이터를 활용해 리뷰에서 메뉴 문장만을 추출합니다

import re
from kiwipiepy import Kiwi
kiwi = Kiwi()

def remove_bracket_contents(menu_names):
    cleaned_list = []

    # 괄호 안의 내용을 모두 지우는 정규 표현식 패턴
    pattern = r'[\[\(].*?[\]\)]'

    for menu_name in menu_names:

        # 패턴을 사용하여 텍스트에서 괄호 안의 내용을 모두 지움
        cleaned_text = re.sub(pattern, '', menu_name)

        if cleaned_text is not None:
            cleaned_list.append(cleaned_text)

    return cleaned_list

def split_menu(menu_names):
    menu_sample_list = []
    menu_dict = []

    for menu_name in menu_names:
        org_menu_dict = {}
        split_result = re.split(r'[+\-\s]+', menu_name)

        # 원본 메뉴명에 대해 딕셔너리 키 생성
        if menu_name not in org_menu_dict:
            org_menu_dict[menu_name] = []  # 초기 리스트 생성


        # 띄어쓰기 단위로 잘라진 메뉴 이름 저장시키기
        menu_name_list = []

        tokened_blank_menu_list = []
        for result in split_result:
            if any(char.isdigit() for char in result):
                # 숫자일 경우 무시
                continue
            else:
                # " ", ""으로 조인시킬 메뉴 네임 리스트(임시적인 리스트)
                menu_name_list.append(result)

                # 얻어진 메뉴명을 토크나이즈 해주고 그것도 메뉴 리스트에 포함
                kiwi_tokens = kiwi.tokenize(result)
                tokened_blank_menu = " ".join([token.form for token in kiwi_tokens])
                tokened_blank_menu_list.append(tokened_blank_menu)

            tokened_blank_menu_join = " ".join(tokened_blank_menu_list)
            # 딕셔너리에 원본 메뉴명의 변형본 추가와 메뉴 샘플리스트에 넣기
            org_menu_dict[menu_name].append(tokened_blank_menu_join)
            menu_sample_list.append(tokened_blank_menu_join)

        menu_dict.append(org_menu_dict)

        
        blank_menu = " ".join(menu_name_list)
        contiuous_menu = "".join(menu_name_list)

        menu_sample_list.append(blank_menu)
        menu_sample_list.append(contiuous_menu)

        # 원본 메뉴 명의 변형을 딕셔너리에 저장
        org_menu_dict[menu_name].append(blank_menu)
        org_menu_dict[menu_name].append(contiuous_menu)
        org_menu_dict[menu_name] = list(set(org_menu_dict[menu_name]))

        # +, -, 공백 단위로 띄워준 메뉴 이름을 리스트에 넣어줌
        # split_result = re.split(r'[+\-\s]+', menu_name)
        # menu_sample_list.extend(split_result)

        # for result in split_result:
        #     # 얻어진 메뉴명을 토크나이즈 해주고 그것도 메뉴 리스트에 포함
        #     kiwi_tokens = kiwi.tokenize(result)
        #     menu_sample_list.extend([token.form for token in kiwi_tokens])

    # 리스트를 set으로 변환하여 중복 제거
    menu_sample_list = set(menu_sample_list)
    # set을 다시 리스트로 변환
    menu_sample_list = list(menu_sample_list)
    
    return menu_sample_list, menu_dict

def make_menu_list(df_menu):
    # 'rst_name'을 기준으로 그룹지어서 'menu_name' 열의 모든 값을 리스트로 모으기
    grouped = df_menu.groupby('restaurant_id').agg({
        'name': lambda x: list(x)
    })
    
    grouped['preprocessed_menu_name'] = grouped['name'].apply(remove_bracket_contents)

    # 'menu_name' 열의 각 값들을 띄어쓰기 단위로 분할하여 리스트로 만들기
    grouped[['menu_name_split', 'org_menu_dict']] = grouped['preprocessed_menu_name'].apply(split_menu).apply(pd.Series)

    return grouped


In [23]:
menu2input = make_menu_list(menu_sample)

In [24]:
menu2input[:2]

,name,preprocessed_menu_name,menu_name_split,org_menu_dict
restaurant_id,,,,
54,"[판모밀, 점심정식, 새우튀김, 전복죽, 알탕, 생대구탕 지리, 코마찌 사시미코스]","[판모밀, 점심정식, 새우튀김, 전복죽, 알탕, 생대구탕 지리, 코마찌 사시미코스]","[판 모밀, 점심정식, 알탕, 생대구탕 지리, 점심 정식, 전복죽, 새우 튀김, 코...","[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '..."
59,"[제주 모듬 근고기 600g, 제주특갈매기살 1인분, 제주 통삼겹살 1인분, 제주 ...","[제주 모듬 근고기 600g, 제주특갈매기살 1인분, 제주 통삼겹살 1인분, 제주 ...","[제주 모듬, 제주 통 삼겹살, 제주 모듬 근고기, 제주 특 항정살, 제주 모듬 그...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주..."


In [26]:
# 가공된 리뷰, 메뉴 테이블을 조인하고 menu sentece 선별

# 'restaurant_id' 컬럼을 기준으로 조인
rev2input = rev2input[['restaurant_id', 'review_sentence_split']]

keybert_input = pd.merge(rev2input, menu2input, on='restaurant_id')
keybert_input = keybert_input[['restaurant_id', 'review_sentence_split', 'menu_name_split', 'org_menu_dict']]

In [33]:
import ast

try:
    keybert_input["review_sentence_split"] = keybert_input["review_sentence_split"].apply(ast.literal_eval)
except:
    pass

def filter_menu_sentences(row):
    # review_sentence_split의 각 문장에 대해
    filtered_sentences = []
    
    sentences = row['review_sentence_split']
    for sentence in sentences:
        # sentence가 menu_name_split의 단어들을 포함하는지 확인
        if any(menu_word in sentence for menu_word in row['menu_name_split']):
            filtered_sentences.append(sentence)
    return filtered_sentences

# 데이터프레임 df에서 각 행에 대해 함수 적용
keybert_input['menu_sentence'] = keybert_input.apply(filter_menu_sentences, axis=1)

In [35]:
keybert_input.head(5)

,restaurant_id,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence
0,54,"[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 아니 회뿐만 ...","[판 모밀, 점심정식, 알탕, 생대구탕 지리, 점심 정식, 전복죽, 새우 튀김, 코...","[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '...","[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코..."
1,59,"[자주 가는 맛집에요 고기 두툼하고 퀄리티 좋구요, 정 말 맛잇어요 메뉴 다양한데 ...","[제주 모듬, 제주 통 삼겹살, 제주 모듬 근고기, 제주 특 항정살, 제주 모듬 그...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이..."
2,63,[가격이 저렴해서 좋아요 양배추 샐러드는 포장은 안 되고 따로 금액 추가해도 안된다...,"[옛날통닭 두마리, 깐풍통닭, 눈꽃 치즈 순 살 치킨, 양념통닭, 깐풍 닭강정, 양...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ..."
3,70,[음식에 미나리 를 깔아주고 메뉴에 미나리도 있었는데 들기름이나 침기름으로 무친건 ...,"[생 합 탕, 생합탕, 갑오징어 숙 회, 갑오징어숙회, 삼 합, 삼합]","[{'생합탕': ['생 합 탕', '생합탕']}, {'갑오징어숙회': ['갑오징어 ...","[홍어삼합 강추합니다, 삼합 추천합니다 .]"
4,82,[금요일에는 손님이 많고 그 때 말고는 괜찮은거 같아 요 손님 많은날 가서 곱창 나...,"[간 / 천엽, 소대창, 염통, 소곱창전골소, 소 막창, 소막창, 소곱창, 소 곱창...","[{'소곱창': ['소곱창', '소 곱창']}, {'소막창': ['소막창', '소 ...",[]


In [38]:
keybert_input.to_csv(path + "keybert_input.csv", encoding='utf-8-sig', index=False)

### inference

In [39]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel
from keybert import KeyBERT
import ast

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_

In [40]:
path = "./data/"

menu_sample = pd.read_csv(path + "menu_sample.csv", encoding='utf-8-sig')
rev_sample = pd.read_csv(path + "review_sample.csv", encoding='utf-8-sig')

keybert_input = pd.read_csv(path + "keybert_input.csv", encoding='utf-8-sig')

In [41]:
# functions
from kiwipiepy import Kiwi

kiwi = Kiwi()

def adverb_remover(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, len_token in result[0][0]:
        if (
            len_token != 1
            and pos.startswith("J") == False
            and pos.startswith("E") == False
            and pos.startswith("MAJ") == False
        ):
            results.append(token)
    return results


In [42]:
# KeyBERT 로드. (KoBERT 사용)

model = BertModel.from_pretrained("skt/kobert-base-v1")
# KeyBERT 모델 초기화 (skt의 Kobert 사용)
kw_model = KeyBERT(model)

In [44]:
keybert_input.head(5)

,restaurant_id,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence
0,54,"['1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요', '아니 회...","['판 모밀', '점심정식', '알탕', '생대구탕 지리', '점심 정식', '전복...","[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '...","['1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요', '코마찌사..."
1,59,"['자주 가는 맛집에요 고기 두툼하고 퀄리티 좋구요', '정 말 맛잇어요 메뉴 다양...","['제주 모듬', '제주 통 삼겹살', '제주 모듬 근고기', '제주 특 항정살',...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","['제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ .', '제주도에서나 볼법한 메..."
2,63,['가격이 저렴해서 좋아요 양배추 샐러드는 포장은 안 되고 따로 금액 추가해도 안된...,"['옛날통닭 두마리', '깐풍통닭', '눈꽃 치즈 순 살 치킨', '양념통닭', '...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","['순살후라이드 먹었어요', '양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ..."
3,70,['음식에 미나리 를 깔아주고 메뉴에 미나리도 있었는데 들기름이나 침기름으로 무친건...,"['생 합 탕', '생합탕', '갑오징어 숙 회', '갑오징어숙회', '삼 합', ...","[{'생합탕': ['생 합 탕', '생합탕']}, {'갑오징어숙회': ['갑오징어 ...","['홍어삼합 강추합니다', '삼합 추천합니다 .']"
4,82,['금요일에는 손님이 많고 그 때 말고는 괜찮은거 같아 요 손님 많은날 가서 곱창 ...,"['간 / 천엽', '소대창', '염통', '소곱창전골소', '소 막창', '소막창...","[{'소곱창': ['소곱창', '소 곱창']}, {'소막창': ['소막창', '소 ...",[]


In [45]:
# keybert돌리고 키워드 리턴하는 함수 (!! -- ngram은 무조건 켜야함 -- !!)
def extract_keywords_with_candidate(document,candidate,top_n=20):
    # 주어진 리뷰들의 문장 리스트에서 각 문장별로 키워드를 추출하여 출력
    keywords = kw_model.extract_keywords(
        document,
        keyphrase_ngram_range=(1,3),  # 단어 n-gram 범위
        stop_words=None,  # 불용어
        # use_maxsum=False,
        use_mmr=True,
        diversity=0.9,  # 다양성
        top_n=top_n,
        # highlight=True,
        candidates=candidate
    )  # 상위 n개 키워드

    return keywords

# 데이터 형식을 input text에 맞게 변환하는 함수 제작 (for문 안에서 돌던거를 위로 뺐음)
def doc_to_input_text(review_doc):
    # 식당의 문장 리스트 추출
    input_text = ".".join(review_doc)
    input_text = adverb_remover(input_text)
    input_text=" ".join(input_text)

    return input_text


from collections import defaultdict

# df1의 각 main_menu가 어떤 카테고리에 속하는지 확인
def sub_to_org(menu, submenu_to_menu):
    for key, org_menuname in submenu_to_menu.items():
        if menu == key:
            return org_menuname
    
    return menu  # 카테고리를 찾지 못한 경우


def duplicate_removed_list(duplicated_list):
    # [[menu1, score1], [menu1, score2]]
    # 각 메뉴 항목에 대한 스코어를 저장할 defaultdict 생성
    menu_scores = defaultdict(list)

    # 데이터를 defaultdict에 저장하여 그룹화
    for menu, score in duplicated_list:
        menu_scores[menu].append(score)

    # 각 메뉴 항목의 스코어 평균을 계산
    averaged_scores = []
    for menu, scores in menu_scores.items():
        average_score = sum(scores) / len(scores)
        averaged_scores.append([menu, average_score])
    # [[menu1, avg_score1], [menu2, avg_score2]]
    
    return averaged_scores

    

In [62]:
# 로드해서 string으로 리스트를 다시 파이썬 리스트로 변환
try:
    keybert_input["org_menu_dict"] = keybert_input["org_menu_dict"].apply(ast.literal_eval)
except:
    pass

try:
    keybert_input["menu_sentence"] = keybert_input["menu_sentence"].apply(ast.literal_eval)
except:
    pass

try:
    keybert_input["menu_name_split"] = keybert_input["menu_name_split"].apply(ast.literal_eval)
except:
    pass

try:
    keybert_input["review_sentence_split"] = keybert_input["review_sentence_split"].apply(ast.literal_eval)
except:
    pass


# input text 열 만들기
keybert_input['input_text'] = keybert_input['menu_sentence'].apply(doc_to_input_text)

In [63]:
keybert_input

,restaurant_id,review_sentence_split,menu_name_split,org_menu_dict,menu_sentence,input_text
0,54,"[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 아니 회뿐만 ...","[판 모밀, 점심정식, 알탕, 생대구탕 지리, 점심 정식, 전복죽, 새우 튀김, 코...","[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '...","[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코...",코마찌 사시미 코스 시키 진짜 싱싱 살살 코마찌사시미코스 부르 가족 모임 코마찌 결...
1,59,"[자주 가는 맛집에요 고기 두툼하고 퀄리티 좋구요, 정 말 맛잇어요 메뉴 다양한데 ...","[제주 모듬, 제주 통 삼겹살, 제주 모듬 근고기, 제주 특 항정살, 제주 모듬 그...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이...",제주도 웨이팅 맛있 ㅎㅎ .. 제주도 메뉴 다른 메뉴 .. 워커힐 숙박 호텔식 검색...
2,63,[가격이 저렴해서 좋아요 양배추 샐러드는 포장은 안 되고 따로 금액 추가해도 안된다...,"[옛날통닭 두마리, 깐풍통닭, 눈꽃 치즈 순 살 치킨, 양념통닭, 깐풍 닭강정, 양...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...",후라이드 양념 닭강정 아이 .. 옛날 통닭 강추
3,70,[음식에 미나리 를 깔아주고 메뉴에 미나리도 있었는데 들기름이나 침기름으로 무친건 ...,"[생 합 탕, 생합탕, 갑오징어 숙 회, 갑오징어숙회, 삼 합, 삼합]","[{'생합탕': ['생 합 탕', '생합탕']}, {'갑오징어숙회': ['갑오징어 ...","[홍어삼합 강추합니다, 삼합 추천합니다 .]",홍어 강추 추천
4,82,[금요일에는 손님이 많고 그 때 말고는 괜찮은거 같아 요 손님 많은날 가서 곱창 나...,"[간 / 천엽, 소대창, 염통, 소곱창전골소, 소 막창, 소막창, 소곱창, 소 곱창...","[{'소곱창': ['소곱창', '소 곱창']}, {'소막창': ['소막창', '소 ...",[],
...,...,...,...,...,...,...
87,3076,"[저녁모임으로 스페셜 저녁은 술자리 위주인가봐, 요 아저씨손님들이고 목소리 엄청 크...","[도토리묵사발, 번데기탕, 쥐포구 이, 쥐포구이, 연유토마토, 도토리묵 사발, 연유...","[{'도토리묵사발': ['도토리묵 사발', '도토리묵사발']}, {'쥐포구이': [...",[],
88,3096,"[국물 닭발이랑 닭똥집 튀김 먹었는데 음식 궁합이 좋고 있게 잘먹었습니다 ., 안...","[똥 집 튀김, 닭껍질 튀김, 가라아게탕수육, 돼지고기, 제육볶음, 국물 떡볶이, ...","[{'나가사끼짬뽕탕': ['나가사끼짬뽕탕', '나가사끼 짬뽕 탕']}, {'해물누룽...","[국물 닭발이랑 닭똥집 튀김 먹었는데 음식 궁합이 좋고 있게 잘먹었습니다 ., 맨...",국물 닭발 닭똥집 튀김 음식 궁합 .. 맨날 가라아게 탕수육 단체 너무 해물 누룽지...
89,3101,"[깔끔하구 분위기 괜찮아요, 다트 하기두 조아용 ., 늘 와도 언제나 와도 건대는 ...","[퀘사디아, 감자 튀김, 세계맥주, 먹태구이, 위스키 샷, 감자튀김, 와인, 먹태구...","[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마...",[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,친절 다트 알리 안주 칵테일 가성비 보장 스타 사랑 .. 감자튀김 치즈 스틱 너무 ...
90,3110,"[칵테일 맛이 죠아요 ., 여기는요 진짜 다 맛있는데요, 정말 정말 타다끼는 미쳤...","[, 후라이드, 베이컨 체 다 치즈 피자, 감바스 알 어 히 요, 베이컨체다치즈피자...","[{'고르곤졸라피자': ['고르곤졸라 피자', '고르곤졸라피자']}, {'페파로니피...","[칵테일 맛이 죠아요 ., 여기는요 진짜 다 맛있는데요, 정말 정말 타다끼는 미쳤...",칵테일 죠아 이 .. 여기 진짜 정말 정말 타다끼 미치 일단 궁금하 체고 .. 맛있...


In [74]:
keybert_input.iloc[2, 5]

'후라이드 양념 닭강정 아이 .. 옛날 통닭 강추'

In [85]:
import warnings

# 'UserWarning' 유형의 경고를 숨김
warnings.filterwarnings('ignore', category=UserWarning, message="Upper case characters found in vocabulary while 'lowercase' is True")


rst_name_list, menu_name_list, keybert_scores = [], [], []

output=[]
for i in range(len(keybert_input)):
    # 식당의 문장 리스트 추출
    
    input_text = keybert_input.loc[i, 'input_text']
    menu_candidates=keybert_input.loc[i, 'menu_name_split']
    org_menu_dict=keybert_input.loc[i,'org_menu_dict']

    review = keybert_input.loc[i,'menu_sentence']
    rst_name = keybert_input.loc[i, 'restaurant_id']

    results = extract_keywords_with_candidate(input_text,menu_candidates)
    # print(results)

    # 메뉴 이름을 키로 하고 카테고리를 값으로 하는 새로운 딕셔너리 생성
    submenu_to_menu = {}
    for menu_dict in org_menu_dict:
        for key, values in menu_dict.items():
            for value in values:
                submenu_to_menu[value] = key

    menu_score = []
    if results:
        # 각 결과에 대해 df에 들어갈 각 행으로 변환
        for result in results:
            name, score = result[0], result[1]
            name = sub_to_org(name, submenu_to_menu)
            menu_score.append([name, score])
            
        # 같은 원본 메뉴 이름을 가진 것들의 중복 제거
        menu_score = duplicate_removed_list(menu_score)
        for menu, score in menu_score:
            print(f"restaurant id = {rst_name}")
            print((menu, score), end=", ")
            rst_info = []

            rst_info.append(rst_name)
            rst_info.append(menu)
            rst_info.append(score)
            rst_info.append(review)
            rst_info.append(org_menu_dict)

            output.append(rst_info)

        del [[rst_info]]
        print()
        
    # else:
    #     print(f"PASS")
    print()

('코마찌 사시미코스', 0.6926333333333333), 
('제주 모듬 근고기 600g', 0.403), 
('옛날통닭 두마리', 0.6805), ('양념닭강정', 0.6021), 
PASS
PASS
PASS
('평양냉면', 0.5069), ('비빔냉면', 0.3652), ('특수모듬제육', 0.1419), 
('아롱사태전골', 0.7719), ('육회+육사시미', 0.2818), 
('김치전', 0.6742), ('콩국수', 0.3191), ('멸치칼국수', 0.2494), 
PASS
('만두 ', 0.5464), ('만두국', 0.4887), ('갈비탕', 0.4377), ('함흥 물냉면+열무+무채', 0.38), 
PASS
PASS
PASS
('비빔밥', 0.723), ('돌솥비빔밥', 0.6757), ('제육볶음', 0.6433), ('오징어볶음', 0.6284), ('김치찌개', 0.6211), 
PASS
PASS
('육회비빔밥', 0.3779), 
('매콤 왕 등갈비)', 0.4988), ('바베큐 왕 등갈비)', 0.3041), 
PASS
('닭볶음탕', 0.8234), 
('들깨 삼계탕', 0.5792999999999999), ('해물 청양 부추전', 0.2632), 
('쫄면', 0.6127), ('오므라이스', 0.5958), ('돈까스', 0.5062), ('우동', 0.4164), 
('고기 반미 Banh My Thit', 0.5637), ('오징어 튀김', 0.502), ('치킨 샐러드 NOM Ga', 0.5388), ('치킨 샤브샤브 Lau Ga', 0.4907), ('월남쌈 3개 Goi Cuon', 0.4863), ('새우 버터마늘 볶음 Tom Sot Bo Toi', 0.43125), ('반미 바대', 0.428), ('짜조 RAM', 0.4136), 
('오마카세 3꼬치 코스', 0.6134999999999999), 
PASS
('막국수', 0.2455), ('닭갈비', 0.1526), 
('올리브페퍼로니베이글', 0.415

Sentiment Penalty

In [92]:
cols = ["restaurant_id", "main_menu", "keybert_score", "menu_reviews" ,"org_menu_dict"]

keybert_output=pd.DataFrame(output,columns=cols)
keybert_output

,restaurant_id,main_menu,keybert_score,menu_reviews,org_menu_dict
0,54,코마찌 사시미코스,0.692633,"[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코...","[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '..."
1,59,제주 모듬 근고기 600g,0.403000,"[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주..."
2,63,옛날통닭 두마리,0.680500,"[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',..."
3,63,양념닭강정,0.602100,"[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',..."
4,183,평양냉면,0.506900,[신상 평양냉면 집은 최대한 가려고 노력하 는데 검색하다보니 못보던 곳이 생겨 바로...,"[{'평양냉면': ['평양 냉면', '평양냉면']}, {'한우 어복쟁반 대': ['..."
...,...,...,...,...,...
176,3096,국물 떡볶이,0.332100,"[국물 닭발이랑 닭똥집 튀김 먹었는데 음식 궁합이 좋고 있게 잘먹었습니다 ., 맨...","[{'나가사끼짬뽕탕': ['나가사끼짬뽕탕', '나가사끼 짬뽕 탕']}, {'해물누룽..."
177,3101,칵테일,0.255800,[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,"[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마..."
178,3101,치즈플레터,0.198900,[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,"[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마..."
179,3101,감자튀김,0.036300,[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,"[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마..."


In [98]:
keybert_output.to_csv(path+"keybert_output.csv", encoding='utf-8-sig', index=False)

In [99]:
# import library
import torch
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

data = keybert_output
# data.head()

In [100]:
# main menu로 filtered_org_menu_dict 생성
def filter_org_menu_dict(row):
    # 문자열로 표기된 딕셔너리 실제 딕셔너리 리스트로 변환
    try:
        org_menu_dicts = ast.literal_eval(row['org_menu_dict'])
    except:
        org_menu_dicts = row['org_menu_dict']
    main_menu = row['main_menu']

    # key에 main_menu 포함하는 org_menu_dicts만 필터링
    filtered_org_menu_dicts = []
    for menu_dict in org_menu_dicts:
        for key, variations in menu_dict.items():
            if main_menu == key:
                filtered_org_menu_dicts.append(menu_dict)
                break
    
    return filtered_org_menu_dicts


# filtered_org_menu_dict로 filtered_reviews 생성
def filter_reviews_by_menu(row):
    # 리뷰 문자열을 리스트로 변환
    try:
        reviews = ast.literal_eval(row['menu_reviews'])
    except:
        reviews = row['menu_reviews']
    
    # filtered_org_menu_dict로부터 추출된 모든 메뉴(value) 추출
    menu_variations = set()
    for menu_dict in row['filtered_org_menu_dict']:
        for variations in menu_dict.values():
            menu_variations.update(variations)
    
    # menu_variations 포함하는 리뷰만 필터링

    filtered_reviews = [review for review in reviews if any(menu_variation in review for menu_variation in menu_variations)]
    
    return filtered_reviews

# filter_org_menu_dict 적용
data['filtered_org_menu_dict'] = data.apply(filter_org_menu_dict, axis=1)

# filter_reviews_by_menu 적용
data['filtered_reviews'] = data.apply(filter_reviews_by_menu, axis=1)

# 결과 출력
data[['main_menu', 'org_menu_dict', 'filtered_org_menu_dict', 'menu_reviews', 'filtered_reviews']].head()


,main_menu,org_menu_dict,filtered_org_menu_dict,menu_reviews,filtered_reviews
0,코마찌 사시미코스,"[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '...","[{'코마찌 사시미코스': ['코마찌', '코마찌사시미코스', '코마찌 사시미 코스...","[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코...","[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코..."
1,제주 모듬 근고기 600g,"[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이...","[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이..."
2,옛날통닭 두마리,"[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","[{'옛날통닭 두마리': ['옛날통닭 두마리', '옛날 통닭', '옛날 통닭 두 마...","[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...",[옛날 통닭 강추 .]
3,양념닭강정,"[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","[{'양념닭강정': ['양념닭강정', '양념 닭강정']}]","[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...",[양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 .]
4,평양냉면,"[{'평양냉면': ['평양 냉면', '평양냉면']}, {'한우 어복쟁반 대': ['...","[{'평양냉면': ['평양 냉면', '평양냉면']}]",[신상 평양냉면 집은 최대한 가려고 노력하 는데 검색하다보니 못보던 곳이 생겨 바로...,[신상 평양냉면 집은 최대한 가려고 노력하 는데 검색하다보니 못보던 곳이 생겨 바로...


In [112]:
data.loc[12, 'filtered_reviews']

['냉면이 갑자기 땡겨서 먹으러갔는데  양도 많고 맛있었어요 만두도 시켰는데 만두도 맛있음  전에는 배달로 시켜먹어본적 있었어요',
 '아이도잘먹고 만두도 맛있네요',
 '만두도 큼지막하니 맛있어요',
 '리뷰쓰면 만두 반판이니까 다들 꼭 드셔보세용 .',
 '배고파서 서성이다 만두국도 있다는 간판보고 들어왔습니다 친절하시고 만두소도 많이들어가있고 떡도 쫀득해요',
 '진짜 너무맛있어요ㅜㅜ 국물도 최고고 열무김치도 최고인네 만두도 진짜 jmt 근처이신분들은 꼭 방문해보세요',
 '물냉 비냉 만두만두 사진을 안찍었네요',
 '근처에 냉면집이없었는데 ㅠㅠ  새로 오픈해서 와이프랑 같이 방문했는데 저희는 물냉 하나 비냉 하나 만두하나 시켯어요 와이프가 냉면먹더니 내가먹었던건 함흥냉면이아니였구나 그러면서 이제알았다고 하네요',
 '거기에 고기도 같이 나왔는데 이것도 너무 부드럽고맛잇는데 일단 만두도 대박이예요',
 '새로생긴곳이라 방문해서 물냉비냉만두 시켰는데 넘 맛있어서 갈비찜도 포장했어요ㅎㅎ',
 '만두도 짱커요 .',
 '서비스로 군만두도 주셨는데 너무 잘 먹었습니다 .',
 '비냉 만두 시켰어요',
 '김치랑 반찬도 깔끔하고 맛있고 만두도 맛있어요',
 '회냉면이랑 만두 먹었어요',
 '냉면 양념이 자극적이지 않고 만두피도 얇아서 더 맛있게 먹었습니다 ㅎㅎ',
 '만두는 더 맛있어요 .',
 '새로 개업했는데 맛있고요 깨끗하고요 물냉 비냉 갈비탕 만두 먹었는데 다 맛있어요',
 '만두도 오왕왕 맛있어요']

In [118]:
# load model

tokenizer = AutoTokenizer.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("jaehyeong/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

c:\Users\MOONSOO\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [119]:
# filtered_reviews로 senti_score 계산

# 감성분석 함수 (0~1 사이 값으로 출력)
def get_sentiment_score(sentence):
    result = sentiment_classifier(sentence)[0]
    return result['score'] if result['label'] == '1' else 1-(result['score'])


# Sentiment Penalty(SP Penalty) 계산 함수

'''
# 먼저 alpha(우리 표현으로는 lambda) 지정해줘야 함
alpha = 1
나중에 하는 버전으로 코드 변경함
'''

def calculate_SP(sentences):
    positive_count = sum(1 for sentence in sentences if sentiment_classifier(sentence)[0]['label'] == '1')
    total_reviews = len(sentences)
    if total_reviews == 0:
        return 0  # 예외 처리: 리뷰가 없는 경우
    exponent_value = 1 - ((1 + total_reviews) / (1 + positive_count))
    # return (np.e + alpha) ** exponent_value
    return exponent_value


# 'reviews' 열의 문장 리스트를 사용하여 평균 감성 점수 계산
data['exponent_value'] = data['filtered_reviews'].apply(calculate_SP)

In [122]:
# total_score 열 계산

alpha = 1.2 # 알파 적용은 아래 두 식 중 양자택일

# alpha 반영하여 e^(alpha*exponent_value) 로 계산하는 식
data['Sentiment_Penalty'] = np.exp(data['exponent_value']*alpha)

data['total_score'] = data['keybert_score'] * data['Sentiment_Penalty']

# rst_name 열의 값이 같은 식당 별로 total_score가 높은 순으로 정렬
sorted_data = data.sort_values(by=['restaurant_id','total_score'], ascending=[True, False])

In [123]:
sorted_data

,restaurant_id,main_menu,keybert_score,menu_reviews,org_menu_dict,filtered_org_menu_dict,filtered_reviews,exponent_value,Sentiment_Penalty,total_score
0,54,코마찌 사시미코스,0.692633,"[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코...","[{'판모밀': ['판 모밀', '판모밀']}, {'점심정식': ['점심정식', '...","[{'코마찌 사시미코스': ['코마찌', '코마찌사시미코스', '코마찌 사시미 코스...","[1인 코마찌 사시미코스시켰는데 회들이 진짜 싱싱하 고 살살 녹아요, 코마찌사시미코...",0.000000,1.000000,0.692633
1,59,제주 모듬 근고기 600g,0.403000,"[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","[{'제주 모듬 근고기 600g': ['제주 모듬', '제주 모듬 근고기', '제주...","[제주도에서 웨이팅해서 먹은 맛집보다 맛있어요ㅎㅎ ., 제주도에서나 볼법한 메뉴들이...",-0.142857,0.842460,0.339512
2,63,옛날통닭 두마리,0.680500,"[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","[{'옛날통닭 두마리': ['옛날통닭 두마리', '옛날 통닭', '옛날 통닭 두 마...",[옛날 통닭 강추 .],0.000000,1.000000,0.680500
3,63,양념닭강정,0.602100,"[순살후라이드 먹었어요, 양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 ., ...","[{'옛날통닭 한마리': ['옛날통닭 한마리', '옛날 통닭', '옛날통닭한마리',...","[{'양념닭강정': ['양념닭강정', '양념 닭강정']}]",[양념 닭강정 안맵다고 하셨는데 아이들 먹기엔 맵네요 .],-1.000000,0.301194,0.181349
4,183,평양냉면,0.506900,[신상 평양냉면 집은 최대한 가려고 노력하 는데 검색하다보니 못보던 곳이 생겨 바로...,"[{'평양냉면': ['평양 냉면', '평양냉면']}, {'한우 어복쟁반 대': ['...","[{'평양냉면': ['평양 냉면', '평양냉면']}]",[신상 평양냉면 집은 최대한 가려고 노력하 는데 검색하다보니 못보던 곳이 생겨 바로...,-0.050000,0.941765,0.477380
...,...,...,...,...,...,...,...,...,...,...
176,3096,국물 떡볶이,0.332100,"[국물 닭발이랑 닭똥집 튀김 먹었는데 음식 궁합이 좋고 있게 잘먹었습니다 ., 맨...","[{'나가사끼짬뽕탕': ['나가사끼짬뽕탕', '나가사끼 짬뽕 탕']}, {'해물누룽...","[{'국물 떡볶이': ['국물떡볶이', '국물', '국물 떡볶이']}]",[국물 닭발이랑 닭똥집 튀김 먹었는데 음식 궁합이 좋고 있게 잘먹었습니다 .],0.000000,1.000000,0.332100
177,3101,칵테일,0.255800,[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,"[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마...",[{'칵테일': ['칵테일']}],[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,0.000000,1.000000,0.255800
178,3101,치즈플레터,0.198900,[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,"[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마...",[{'치즈플레터': ['치즈플레터']}],[치즈플레터를 주문할 거라면 기본으로 제공되는 크래커버터딸기잼과 함께 먹는 걸 추천...,0.000000,1.000000,0.198900
179,3101,감자튀김,0.036300,[갈때 마다 친절하고 다트도 잘 알려주는 곳 안주 및 칵테일 가성비 맛 보장 믿고...,"[{'치즈플레터': ['치즈플레터']}, {'퀘사디아': ['퀘사디아']}, {'마...","[{'감자튀김': ['감자튀김', '감자 튀김']}]",[그리고 감자튀김 치즈스틱 너무 맛있었어요 .],0.000000,1.000000,0.036300


테이블에 매핑

In [126]:
menu_sample = menu_sample.iloc[:, 1:]
menu_sample

,menu_id,price,restaurant_id,name,ranking,created_at,updated_at,image_url
0,492,10000,54,판모밀,NaN,NaN,NaN,NaN
1,493,30000,54,점심정식,NaN,NaN,NaN,NaN
2,494,30000,54,새우튀김,NaN,NaN,NaN,NaN
3,495,15000,54,전복죽,NaN,NaN,NaN,NaN
4,496,15000,54,알탕,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
989,25053,12900,3128,해물치즈떡볶이,NaN,NaN,NaN,NaN
990,25054,13900,3128,돈가스치즈떡볶이,NaN,NaN,NaN,NaN
991,25055,12900,3128,오돌뼈+주먹밥,NaN,NaN,NaN,NaN
992,25056,4900,3128,스팸주먹밥,NaN,NaN,NaN,NaN


In [127]:
# restaurant_id를 기준으로 두 데이터프레임 결합
combined_data = pd.merge(menu_sample, sorted_data, on='restaurant_id')

# menu_sample의 name과 sorted_data의 main_menu 매칭 및 ranking 업데이트
def update_ranking(row, grouped_data):
    if row['name'] in grouped_data['main_menu'].values:
        # main_menu에 해당하는 total_score에 따라 ranking 업데이트
        return grouped_data[grouped_data['main_menu'] == row['name']]['total_score'].iloc[0]
    else:
        return None

# 각 restaurant_id 그룹별로 처리
menu_sample['ranking'] = menu_sample.apply(lambda row: update_ranking(row, combined_data[combined_data['restaurant_id'] == row['restaurant_id']]), axis=1)

# total_score에 따라 순위 부여
menu_sample['ranking'] = menu_sample.groupby('restaurant_id')['ranking'].rank(method='dense', ascending=False)


In [128]:
# 결과 출력
menu_sample

,menu_id,price,restaurant_id,name,ranking,created_at,updated_at,image_url
0,492,10000,54,판모밀,NaN,NaN,NaN,NaN
1,493,30000,54,점심정식,NaN,NaN,NaN,NaN
2,494,30000,54,새우튀김,NaN,NaN,NaN,NaN
3,495,15000,54,전복죽,NaN,NaN,NaN,NaN
4,496,15000,54,알탕,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
989,25053,12900,3128,해물치즈떡볶이,NaN,NaN,NaN,NaN
990,25054,13900,3128,돈가스치즈떡볶이,NaN,NaN,NaN,NaN
991,25055,12900,3128,오돌뼈+주먹밥,1.0,NaN,NaN,NaN
992,25056,4900,3128,스팸주먹밥,NaN,NaN,NaN,NaN


In [ ]:
# ranking 열이 null이 아닌 행들만 필터링
filtered_menu_sample = menu_sample[menu_sample['ranking'].notna()]

filtered_menu_sample